In [2]:
#Install required libraries
!pip install opencv-python matplotlib numpy pillow scikit-learn


In [7]:
import os

base_dir = r"C:\Users\23052\Desktop\PDE 3802-C1\raw"

classes = ["water_bottles", "pens", "mouse"]

for cls in classes:
    folder = os.path.join(base_dir, cls)
    if os.path.exists(folder):
        print(f"\nClass: {cls}")
        print("Number of images:", len(os.listdir(folder)))
        print("Example files:", os.listdir(folder)[:5])
    else:
        print(f"\n⚠ Folder not found: {folder}")



Class: water_bottles
Number of images: 12
Example files: ['image00001.jpg', 'image00001.png', 'image00002.jpg', 'image00003.jpg', 'image00004.jpg']

Class: pens
Number of images: 22
Example files: ['image00001.jpeg', 'image00002.jpeg', 'image00003.jpeg', 'image00004.jpeg', 'image00005.jpeg']

Class: mouse
Number of images: 20
Example files: ['image00001.jpg', 'image00002.jpeg', 'image00003.jpg', 'image00004.jpeg', 'image00005.jpeg']


In [9]:
#Image Processing and cleaning
import os
import cv2

# Define paths
RAW_DIR = r"C:\Users\23052\Desktop\PDE 3802-C1\raw"
PROCESSED_DIR = r"C:\Users\23052\Desktop\PDE 3802-C1\processed"

#Desired Image size
imageSize=(224,224)

# Create processed directory if not exists
if not os.path.exists(PROCESSED_DIR):
    os.makedirs(PROCESSED_DIR)

#Loop through each class folder
for class_name in os.listdir(RAW_DIR):
    class_path = os.path.join(RAW_DIR, class_name)

    # Skip non-folders
    if not os.path.isdir(class_path):
        continue

    print(f"Processing class: {class_name}")

    # Create mirrored folder in processed/
    processed_class_path = os.path.join(PROCESSED_DIR, class_name)
    os.makedirs(processed_class_path, exist_ok=True)

    # Process each image
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)

        # Read image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping (not readable): {img_file}")
            continue

        try:
            # Resize
            resized_img = cv2.resize(img, imageSize)

            # Save to processed/folder
            save_path = os.path.join(processed_class_path, img_file)
            cv2.imwrite(save_path, resized_img)
        except Exception as e:
            print(f"Error processing {img_file}: {e}")

print("\n Processing completed. Images saved to 'dataset/processed'.")


Processing class: mouse
Processing class: notebook
Processing class: paper clips
Processing class: pens
Processing class: scissors
Processing class: stapler
Processing class: water_bottles

 Processing completed. Images saved to 'dataset/processed'.


Image augmentation



In [27]:

import os
import cv2
import numpy as np
import random  

PROCESSED_DIR = r"C:\Users\23052\Desktop\PDE 3802-C1\processed"
AUGMENTED_DIR=r"C:\Users\23052\Desktop\PDE 3802-C1\augmented"

#create augmented folder if not exists
if not os.path.exists(AUGMENTED_DIR):
    os.makedirs(AUGMENTED_DIR)
    
def rotate(image):
    angle= random.choice([15,-15,30,-30])
    # dividing height and width by 2 to get the center of the image
    height, width = image.shape[:2]
    # get the center coordinates of the image to create the 2D rotation matrix
    center = (width/2, height/2)
    # using cv2.getRotationMatrix2D() to get the rotation matrix
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=45, scale=1)
     
    # rotate the image using cv2.warpAffine
    rotated_image = cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    return rotated_image

#flip image function
def flip(image):
    #horizontal flip
    return cv2.flip(image,1)



def change_hue(image):
   
    # Convert BGR to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
     # Access the hue channel
    hue_channel = hsv_image[:, :, 0]
     # Define lower and upper bounds for a specific color (e.g., green)
    lower_green = (40, 50, 50)  # Hue, Saturation, Value
    upper_green = (80, 255, 255)

    # Create a mask for the green color
    mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # Apply the mask to the original image
    result = cv2.bitwise_and(image, image, mask=mask)
    return result
    


def to_grayscale(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

augmentations=[rotate, flip,  change_hue, to_grayscale]

for class_name in os.listdir(PROCESSED_DIR):
    class_path = os.path.join(PROCESSED_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"\nAugmenting class: {class_name}")

    augmented_class_path = os.path.join(AUGMENTED_DIR, class_name)
    os.makedirs(augmented_class_path, exist_ok=True)

    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping unreadable: {img_file}")
            continue

        # Save original (optional)
        cv2.imwrite(os.path.join(augmented_class_path, img_file), img)

        # Create multiple augmented versions
        for i in range(5):  # change number if needed
            aug_func = random.choice(augmentations)
            aug_img = aug_func(img)
            new_filename = f"{os.path.splitext(img_file)[0]}_aug{i}.jpg"
            save_path = os.path.join(augmented_class_path, new_filename)
            cv2.imwrite(save_path, aug_img)

print("\n Augmentation complete. Files saved to 'dataset/augmented/'")

        


Augmenting class: mouse

Augmenting class: notebook

Augmenting class: paper clips

Augmenting class: pens

Augmenting class: scissors

Augmenting class: stapler

Augmenting class: water_bottles

 Augmentation complete. Files saved to 'dataset/augmented/'
